# Word Embeddings

Natürliche Sprachen bieten uns mannigfaltige Möglichkeiten, dieselben Inhalte auf unterschiedliche Art und Weise auszudrücken. Wenn wir mit Texten arbeiten, reicht es daher in der Regel nicht aus, morphologische und syntaktische Strukturen zu betrachten, weil neben der Form und Anordnung der Wörter auch deren Bedeutung eine Rolle spielt. Als zusätzliche und wichtige Ebene kommt hier also die Semantik ins Spiel.

Die Abbildung von Wörtern auf Vektoren erlaubt es uns, mit diesen zu rechnen und zum Beispiel Distanzen oder Ähnlichkeiten zu bestimmen. Embeddings haben den Vorteil, dass sie eine Dimensionsreduktion mit sich bringen und semantische Embeddings sorgen darüber hinaus dafür, dass "verwandte" Wörter einen geringen Abstand voneinander haben.

Wir wollen uns im Folgenden zunächst mit Word2Vec beschäftigen und eine einfache Version des CBOW-Ansatzes selbst implementieren.

Danach schauen wir uns Gensim als Wrapping-Bibliothek für Wordembeddingmodelle an und werfen einen Blick auf Evaluationsmethoden für Embeddings sowie die ihnen inhärenten Biase.

## Aufgabe 1: Word2Vec CBOW
> You shall know a word by the company it keeps.
>
> -- <cite>J. R. Firth</cite>

Auf dem oben zitierten Prinzip beruhen die beiden als Word2Vec bekannt gewordenen Modelle CBOW und Skip Gram, die 2013 von [Tomas Mikolov et al.](https://arxiv.org/abs/1301.3781) bei Google entwickelt wurden.
Erstgenanntes Modell werden wir im Folgenden in einer einfachen Form selbst implementieren.

### 1.1 Trainingsdaten
Die Beschaffung und Aufbereitung von Trainingsdaten ist ein wichtiger Schritt in jeder NLP-Pipeline. Heute drücken wir uns davor und greifen auf das [`text8`-Datenset](http://mattmahoney.net/dc/textdata.html) zurück, das über den [Gensim-Downloader ](https://radimrehurek.com/gensim/downloader.html) heruntergeladen werden kann. Es besteht aus einem Auszug aus der Wikipedia und ist in der Gensim-Version bereits so vorbereitet, dass es als Liste von Wortlisten, den einzelnen Wikipediaartikeln, vorliegt.

Der Datensatz kann [hier](https://drive.google.com/drive/folders/1PKyQnB8Ox7QN7xdC8EjLCXZtDcpw90u3?usp=sharing) heruntergeladen werden. Ladet den Datensatz mit Pickle und wählt einen Teil der Daten (etwa 100 Artikel sollten zu Demonstrationszwecken genügen) als Testdatensatz aus.

In [ ]:
import pickle

data = #TODO
data_sample = #TODO
del data #nicht unbedingt nötig, aber wir verbrauchen eh schon so viel Speicher

### 1.2 Datenvorbereitung
Wir haben einen Datensatz, der aus Listen von Wörtern besteht. Unser Modell soll aber hinterher mit Zahlen hantieren und zwar entweder mit Wortindizes, die jedes Wort im Vokabular über einen eindeutige Nummer referenzierbar machen, oder mit One-hot-Vektoren, die als Labels dienen, mit denen der tatsächliche Output des Modells verglichen werden kann.

In [ ]:
from collections import OrderedDict

# Beim Mapping von Wörtern zu IDs und umgekehrt sollte eine reproduzierbare Reihenfolge sichergestellt werden,
# um das Modell später weitertrainieren und die Embedding-Matrix interpretieren zu können.
# Diese Datenstruktur kann, aber muss nicht, als Basis dienen.
unique_words = OrderedDict.fromkeys(# Liste alle Wörter, die in unserem data_sample vorkommen) 

# Mapping von Wort zu ID
word2id = # TODO

# Mapping von ID zu Wort
id2word = # TODO

# Unser Data Sample, aber mit IDs statt Wörtern 
# [['der', 'hund', 'der', 'bellt'], ['die', 'katz', 'miaut']] => [[0, 1, 0, 2], [3, 4, 5]]
numeric_docs = [[word2id[w] for w in doc] for doc in data_sample]

print('Word to id sample:', list(word2id.items())[:10], '\n')
print('Id to word sample:', list(id2word.items())[:10], '\n')
print('Documents as lists of integers:', numeric_docs[0][:10])


Wir halten einige wichtige Parameter für unser Modell fest. Die Größe des Kontextfensters sowie die Länge der Embeddingvektoren können nach Bedarf angepasst werden. Für unsere Demo wählen wir kleine Werte.

In [ ]:
vocabulary_size = # TODO
embedding_size = 50 # Länge der Embeddingvektoren
window_size = 2 # Größe des Kontextfensters. Wird nach rechts und links angewandt. Gesamter Kontext hier also 4 Wörter.

print('Vocabulary Size:', vocabulary_size)

### 1.3 Generator
Um nicht alle Trainingsdaten auf einmal im Speicher halten zu müssen, schreiben wir uns eine Generatorfunktion, die Batches einer frei wählbaren Größe zurückgibt. Unser Ansatz ist dennoch nicht völlig speicherschonend, weil wir uns die Datengrundlage für die Generierung dieser Batches, nämlich die Integerlisten in `numeric_docs` sehr wohl im Speicher vorhalten. Darüber sehen wir aber großzügig hinweg.

Die Generatorfunktion erzeugt zwei numpy-Arrays der Länge `batch_size`, von denen das eine Listen mit Indizes der Kontexwörter enthält, die der Embedding-Layer als Eingabe erwartet, und das andere die zugehörigen One-Hot-Encodings der Mittelwörter.

Fiktives und vereinfachtes Beispiel:
<pre><code>* Fenstergröße: 1
* Batch-Size: 2
* Wortindizes: 'die': 0, 'ente': 1, 'lacht': 2, 'und': '3, 'quakt': 4 (und damit Vokabulargröße 5)
* Korpus (Auszug): [['die', 'ente', 'lacht', 'und', 'quakt'], ...]


=> Rückgabe: [[0, 2], [1, 3]], [[0, 1, 0, 0, 0], [0, 0, 1, 0, 0]]
</code></pre>

In [ ]:
from keras.preprocessing import sequence
from keras.utils import np_utils
import numpy as np

def generate_context_word_batches(corpus, window_size, vocab_size, batch_size):
    X = []
    Y = []
    current_size = 0
    while True:
        # TODO: Here be dragons
        yield contexts, label_words # zwei numpy arrays


In [ ]:
# Schneller Test
test_batch_size = 3
test_window_size = 2

for i in range(0, 3): 
    x, y = next(generate_context_word_batches(corpus=numeric_docs, window_size=test_window_size, vocab_size=vocabulary_size, batch_size=test_batch_size))
    for j in range(0, test_batch_size):
        print('Context (X):', [id2word[w] for w in x[j]], '-> Target (Y):', id2word[np.argwhere(y[j])[0][0]])  

### 1.4 Definition des Models
Als nächstes definieren wir unser Model. Dazu verwenden wir die Sequential API von Keras.

In [ ]:
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda

#Modelldefinition
cbow = Sequential()
cbow.add(Embedding(input_dim=#TODO, output_dim=#TODO, input_length=#TODO))
cbow.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(embed_size,)))
cbow.add(Dense(#TODO: #Units, activation=#TODO))
cbow.compile(loss=#TODO, optimizer='rmsprop')

# Zusammenfassung
print(cbow.summary())

### 1.5 Training
Jetzt wird es ernst: Wir trainieren unser Modell.
Da wir eine eigene Generatorfunktion verwenden, müssen wir `steps_per_epoch` angeben. Dies ist die Anzahl der Generatoraufrufe pro Epoche. An sich ist es sinnvoll, diesen Wert auf `#samples//batch_size` zu setzen, damit das Modell pro Epoche alle Trainingsdaten sieht, aber weil wir Zeit sparen wollen, wählen wir einen geringeren Wert.

Weil wir unser Modell gerne abspeichern möchten, zum Beispiel, um es später weiter zu trainieren, definieren wir eine Callback-Funktion, die das für uns übernimmt.

In [ ]:
from keras.callbacks import ModelCheckpoint

model_checkpoint = ModelCheckpoint('embeddings.hd5', monitor='loss', verbose=1, save_best_only=True, save_weights_only=False)
   

In [ ]:
epochs = 3
batch_size = 300
# Sollte eigentlich eher etwas in der Art von sum([len(doc) - 2 * window_size for doc in numeric_docs])//batch_size sein, aber s. o.
steps_per_epoch = 500 


cbow.fit_generator(# TODO, callbacks=[#TODO])    

### 1.6 Test
Nachdem wir unser Modell nur sehr kurz und nur auf wenigen Daten trainiert haben, ist davon auszugehen, dass die Ergebnisse nicht optimal sind. Einen kurzen Blick wollen wir dennoch riskieren.

Dazu extrahieren wir zunächst die Gewichte aus dem Embedding-Layer und schauen sie uns auszugsweise an.

In [ ]:
import pandas as pd
from keras.models import load_model

cbow = # TODO: Model laden
embedding_weights = # TODO: Auf Embedding Layer (1. Layer) des Modells zugreifen und dort die Gewichtsmatrix extrahieren

pd.DataFrame(# TODO: Was wollen wir anschauen?, index=list(id2word.values())).head()

Da durch scharfes Hinsehen nicht unmittelbar zu erkennen ist, wie gut unsere Embeddings schon sind, machen wir stichprobenartige Tests. Dazu wählen wir einige Wörter und berechnen für deren Embeddings die Ähnlichkeit mit allen anderen Embedding-Vektoren in unserer Gewichtsmatrix. Anschließend lassen wir uns die fünf ähnlichsten Wörter ausgeben.
Als Distanzmaß wählen wir die Kosinusdistanz, die auf der Kosinusähnlichkeit beruht.

In [ ]:
from sklearn.metrics.pairwise import cosine_distances

sample_terms = ['thus', 'influence', 'bible', 'climate', 'revolution', 'unix', 'term', 'working', 'topic', 'opinion']
sample_embeddings = # TODO

# Berechne die paarweisen Distanzen zwischen Beispielwörtern und Gesamtvokabular
distance_matrix = cosine_distances(# TODO)

In [ ]:
# Zeige die top fünf ähnlichsten Wörter zu unseren Beispielwörtern 
similar_words = {sample_term: [id2word[idx] for idx in distance_matrix[index].argsort()[1:6]] 
                   for index, sample_term in enumerate(sample_terms)}

similar_words

## Aufgabe 2: Gensim als Wrapper für Word2Vec-Modelle
Embedding-Layer begegnen einem in der Praxis in der Tat häufig. In der Regel aber nicht als Bestandteile von reinen Word Embedding-Trainingsmodellen, sondern als erster Layer für Modelle mit anderen Aufgaben. Die Embeddings werden dann entweder mit vorberechneten Werten initalisiert oder werden im Training des Modells für die Downstream-Aufgabe (Textklassifikation, Übersetzung, ...) mittrainiert.

Eine komfortable Möglichkeit, eigene Word2Vec-Modelle zu trainieren, bietet [Gensim](https://radimrehurek.com/gensim/models/word2vec.html), eine Bibliothek, die für diese Modelle auch Wrapper bereitstellt, um komfortabler an die Embeddings zu kommen und mit diesen zu arbeiten.

Wir wollen uns im Folgenden einen kleinen Ausschnitt der Möglichkeiten, die Gensim bietet, anschauen.

### 2.1 Word2Vec-Model laden
Wir werden mit vortrainierten Google-News-Embeddings arbeiten, die [hier](https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit?usp=sharing) heruntergeladen werden können. Die Vektoren haben die Länge 300.
Ladet die Embeddings über den oben angegebenen Link herunter und verwendet gensim, um sie zu laden.

**Hinweis**: Kann einen Moment dauern, bis das Dictionary, das von Wort auf Embedding abbildet, erzeugt ist. Im Zweifel ein ```limit``` angeben und nur die ersten 1,5 Mio. Embeddings laden.

In [ ]:
import gensim

embeddings = # TODO

Aus Spaß an der Freude können wir dann mal schauen, wie gut unser Modell ist bzw. ob es bestimmte von Menschen wahrgenommene Analogien bestätigt.

In [ ]:
from gensim.test.utils import datapath

embeddings.evaluate_word_analogies(datapath("questions-words.txt"), restrict_vocab=30000)

### 2.2 Spaß mit Semantik
Im Folgenden wollen wir uns ein bisschen mit dem Mehrwert beschäftigen, den semantische Embeddings bieten. Weitere Inspiration zum Beispiel [hier](https://www.machinelearningplus.com/nlp/gensim-tutorial/) und in der [Doku](https://radimrehurek.com/gensim/models/keyedvectors.html).

Mit semantischen Vektoren lassen sich zum Beispiel folgende Fragen beantworten:


In [ ]:
# Welche Stadt ist das New York Deutschlands? (Hinweis: 'New_York' ist als Token in den Embeddings enthalten)
print('Das deutsche New York ist: {}\n'.format(# TODO)))

# Was ist Emacs besonders ähnlich?
print('Ähnlichste Begriffe zu "Emacs": {}\n'.format(# TODO))

# Und wie sieht es mit Vim aus?
print('Ähnlichste Begriffe zu "Vim": {}\n'.format(# TODO))

# Wer ist eigentlich der Mozart der Naturwissenschaft?
print('Der Mozart der Naturwissenschaft ist: {}\n'.format(# TODO))

# Welches Wort verhält sich zu 'singing' wie 'burnt' zu 'burning'?
print('burning:burnt wie singing:{}\n'.format(# TODO))

# Sind sich Deutschland und Frankreich ähnlicher oder Deutschland und Kanada?
print('Ähnlichkeit DE, FR: {}'.format(# TODO))
print('Ähnlichkeit DE, CAN: {}'.format(# TODO))

Bei der Interpretation der Ergebnisse ist jedoch Vorsicht geboten: Es werden zwar semantische Beziehungen abgebildet, aber die entsprechen möglicherweise nicht immer den Erwartungen.

Wie ähnlich sind sich zum Beispiel "Leben" und "Tod", "kalt" und "warm", "Norden" und "Süden"?

Sind die Ergebnisse wie erwartet? Warum (nicht)?

In [ ]:
# TODO: Ähnlichkeiten berechnen

Die Embeddings sind nicht neutral, sondern spiegeln die Beziehungen wieder, die sich in den Trainingsdaten finden lassen.

In [ ]:
# Wird Wissenschaft von Frauen oder Männern gemacht?
print('Wissenschaft wird gemacht von: {}'.format(# TODO))
# Sind Mörder eher Schwarze, Weiße oder Asiaten?
print('Mörder sind: {}'.format(# TODO))
# Was bleibt vom Mann, wenn die Intelligenz abgezogen wird?
print('Mann ohne Intelligenz: {}'.format(# TODO))